In [ ]:
import pandas as pd
import numpy as np
import  matplotlib.pyplot as plt

Data Extraction

In [ ]:
#در این بخش دیتا ها را می خوانیم تا ساختار داده ها را بررسی کنیم
input_data=pd.read_csv("titanic.csv")
#input_data
input_data.info()
#input_data.describe()

In [ ]:
#ابتدا چک می کنیم در کدام ستون ها مقدار خالی وجود دارد
input_data.isna().sum()

In [ ]:
#در ستون سن مقادیر خالی ای وجود دارد و برای این که در نتایج مان تاثیر نگذارد آن مقادیر را با میانگین سن مسافران پر می کنیم
input_data.fillna({"Age":input_data["Age"].mean()}, inplace=True)


In [ ]:
#به این سه ستون نیازی نداریم چون هیچ ارتباطی بین آن ها وجود ندارد
input_data.drop(columns=["PassengerId","Name","Ticket"],inplace=True)


In [ ]:
#به دلیل آن که ستون های عددی برای نیاز داریم، ستون جنسیت را به صفر و یک تبدیل می کنیم
input_data["Sex"] = input_data["Sex"].map({"male":1, "female":2})
# هم چنین برای ستون Embarked هم در ابتدا به حرف اول جایی که سوار شدن عدد نگاشت می کنیم تا مشخص شود کدام بندر بیشترین تکرار را دارد تا آن را بجای مادیر خالی در این ستون بگزاریم
input_data.fillna({"Embarked":"S"}, inplace=True)
input_data["Embarked"] = input_data["Embarked"].map({"S":1, "C":2, "Q":3})
#print(np.unique(input_data["Embarked"].dropna(), return_counts=True))



In [ ]:
# در ستون کابین، حرف اول را فقط در نظر می گیریم و اعداد بعد از آن را در نظر نمی گیریم، سپس به بررسی تعداد مسافران در هر کابین می پردازیم
input_data["Cabin"] = input_data["Cabin"].astype(str).str[0]
print(input_data["Cabin"].value_counts(dropna=False))

In [ ]:
# با توجه به نتیجه خروجی بالا، اگر همه موارد خالی را با مد ستون کابین پر کنیم، تحلیل مان اشتباه می شود چون اکثر مسافران می شوند از کابین c و هم چنین ممکن است که واقعا بعضی از افراد کابین نداشته باشند، پس یک دسته بندی جدید به نام unknown با مخفف u درست می کنیم و مقادیر خالی را با آن ها پر می کنیم

input_data["Cabin"] = input_data["Cabin"].astype(str).str[0]

input_data["Cabin"] = input_data["Cabin"].replace("n", "U")

input_data["Cabin"] = input_data["Cabin"].map({
    "A":1, "B":2, "C":3, "D":4, "E":5, "F":6, "G":7, "T":8, "U":0
})


In [ ]:
input_data["Age"]=input_data["Age"].astype(np.uint8)


In [ ]:
#با استفاده از این تابع همه داده ها را بررسی کرده و تا جایی که امکان دارد آن ها را کاهش حجم می دهیم
def auto_data_type(df):
    for col in df.select_dtypes(include=[np.number]).columns:
        min_value = df[col].min()
        max_value = df[col].max()

        if pd.api.types.is_float_dtype(df[col]):
            df[col] = df[col].astype(np.float32)
        elif min_value >= 0:
            if max_value <= 255:
                df[col] = df[col].astype(np.uint8)
            elif max_value <= 65535:
                df[col] = df[col].astype(np.uint16)
            elif max_value <= 4294967295:
                df[col] = df[col].astype(np.uint32)
            else:
                df[col] = df[col].astype(np.uint64)
        else:
            if -128 <= min_value and max_value <= 127:
                df[col] = df[col].astype(np.int8)
            elif -32768 <= min_value and max_value <= 32767:
                df[col] = df[col].astype(np.int16)
            elif -2147483648 <= min_value and max_value <= 2147483647:
                df[col] = df[col].astype(np.int32)
            else:
                df[col] = df[col].astype(np.int64)
    return df

clean_df=auto_data_type(input_data)

#به بررسی مقدار حجم کاهش یا فته می پردازیم تا تغییرات را مشاهده کنیم. همان طور که دیده می شود حدود 88 درصد کاهش حجم رخ داده است.
clean_df.info()


Normalization

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(20,6))

cols = ["Age", "Fare", "SibSp", "Parch"]

for idx, ax in enumerate(axes.ravel()):
    ax.boxplot(clean_df[cols[idx]])
    ax.legend([cols[idx]])

# از پلات زیر می توان نتیجه گرفت که یک نفر بیشترین Fare داشته. پس آن را حذف می کنیم
clean_df = clean_df[clean_df["Fare"] != clean_df["Fare"].max()]